In [1]:
from extract_SATD_comments import _Comment, identify_comments, xml_lexer
import re
from antlr4 import Token, Lexer, InputStream
from lexer.CMakeLexer import CMakeLexer
from lexer.MakefileCommentLexer import MakefileCommentLexer
from lexer.MakefileAmCommentLexer import MakefileAmCommentLexer
from lexer.CPP14Lexer import CPP14Lexer
import pandas as pd

In [2]:
Non_SATD_comments = pd.read_csv('../data/Non_SATD_clones.csv')

In [3]:
def clean_comments(content, comments):
    for comment in comments:
        for line in comment.text.split('\n'):
            content = content.replace(line, '', 1)
    return content

In [4]:
def clean_comments(content, comments):
    i = 0
    for comment in comments:
        for line in comment.get_text().split('\n'):
            content = content.replace(line, '', 1)
    # This is hack to clean undiscoved comment because of Antrl4 cannot read these "wrong" grammar
    lines = [line if len(line.strip()) == 0 else ('' if line.strip()[0] == '#' else line) for line in content.split('\n')]
    content = '\n'.join(lines)
    return content

In [5]:
code_snippets = pd.Series()
threshold = 5

for name, group in Non_SATD_comments.groupby('filePath'):
    file = '/data/satd-clone-2022/' + str(group.iloc[0]['repoIndex']) + '/' + '/'.join(name.split('/')[7:])

    content = open(file, 'r', errors='ignore').read()
    input_stream = InputStream(content)
    if re.match(r'^build\.xml$', file.split('/')[-1]) or re.match(r'^(pom\.xml|maven[123]?\.xml)$', file.split('/')[-1]):
        comments = xml_lexer(file, content)
    elif re.match(r'^([Cc]onfigure.in|[Mm]akefile.in)$', file.split('/')[-1]) or re.match(r'^[Cc]onfigure.ac$', file.split('/')[-1]) or re.match(r'^ac(local|site).m4$', file.split('/')[-1]):
        comments = identify_comments(MakefileCommentLexer(input_stream))
    elif re.match(r'^config.h.in$', file.split('/')[-1]):
        comments = identify_comments(CPP14Lexer(input_stream))
    elif re.match(r'^[Mm]akefile.am$', file.split('/')[-1]):
        comments = identify_comments(MakefileAmCommentLexer(input_stream))
    elif re.findall(r'.cmake', file.split('/')[-1]) or file.split('/')[-1] == 'CMakeLists.txt' or file.split('/')[-1] == 'build.properties':
        comments = identify_comments(CMakeLexer(input_stream))
    else: 
        print(name)
        continue

    content = clean_comments(content, comments)

    lines = content.split('\n')
    
    for index, row in group.iterrows():
        start_line = row['startLine']
        upper_code = []
        while start_line >= 1 and len(upper_code) < threshold:
            code = lines[start_line - 1]
            if code.strip() != '': upper_code.append(code)
            start_line -= 1
        upper_code.reverse()

        end_line = row['endLine']
        bottom_code = []
        while end_line <= len(lines) and len(bottom_code) < threshold:
            code = lines[end_line - 1]
            if code.strip() != '': bottom_code.append(code)
            end_line += 1
        code_snippets.at[index] = '\n'.join(upper_code + bottom_code)


line 1098:41 token recognition error at: '" >&AS_MESSAGE_LOG_FD\n   (exit $ac_status); }])\n\nm4_include([acinclude.m4])\n'
line 4340:30 token recognition error at: '"stamp-h\$am_indx\n    ;;\n  esac\n  am_indx=\`expr \$am_indx + 1\`\ndone])\n])]) # AM_CONFIG_HEADER\n\n# _AM_DIRNAME(PATH)\n# -----------------\n# Like AS_DIRNAME, only do it during macro expansion\nAC_DEFUN([_AM_DIRNAME],\n       [m4_if(m4_regexp([$1], [^.*[^/]//*[^/][^/]*/*$]), -1,\n\t      m4_if(m4_regexp([$1], [^//\([^/]\|$\)]), -1,\n\t\t    m4_if(m4_regexp([$1], [^/.*]), -1,\n\t\t\t  [.],\n\t\t\t  m4_patsubst([$1], [^\(/\).*], [\1])),\n\t\t    m4_patsubst([$1], [^\(//\)\([^/].*\|$\)], [\1])),\n\t      m4_patsubst([$1], [^\(.*[^/]\)//*[^/][^/]*/*$], [\1]))[]dnl\n]) # _AM_DIRNAME\n\n# Add --enable-maintainer-mode option to configure.\n# From Jim Meyering\n\n# serial 1\n\nAC_DEFUN([AM_MAINTAINER_MODE],\n[AC_MSG_CHECKING([whether to enable maintainer-specific portions of Makefiles])\n  dnl maintainer-mode is disabled by 

line 1714:16 token recognition error at: '"\n])\n'
line 247:52 token recognition error at: '")\n\nAC_CONFIG_FILES([\n  Makefile\n  doc/Makefile\n  libusb.spec\n  libusb-config\n  libusb.pc\n  README\n  INSTALL.libusb\n  usb.h\n  Doxyfile\n])\nAC_CONFIG_COMMANDS([default],[[chmod +x libusb-config]],[[]])\nAC_OUTPUT\n\n'
line 245:80 token recognition error at: '"; exit 1\n\ntarball: check_VERSION check-submods check_all_committed check_changelogs_up2date distclean doc configure .filelist\n\trm -f drbd-utils-$(VERSION)\n\t$(LN_S) . drbd-utils-$(VERSION)\n\tfor f in $$(<.filelist) ; do [ -e $$f ] && continue ; echo missing: $$f ; exit 1; done\n\ttar --owner=0 --group=0 -czf - -T .filelist > drbd-utils-$(VERSION).tar.gz\n\trm drbd-utils-$(VERSION)\n\nifdef RPMBUILD\ndrbd.spec: drbd.spec.in configure\n\t./configure --enable-spec\nendif\n\n.PHONY: selinux\nselinux:\n\t$(MAKE) -C selinux\n\nifneq ($(FORCE),1)\ndockerimage: debrelease\nelse\ndockerimage:\nendif\n\tdocker build -t $(DOCKERREGPAT

line 598:49 token recognition error at: '" to perform build as well\n#\ncheck: all\n\t$(MAKE) $(AM_MAKEFLAGS) check-recursive\n'
line 7790:47 token recognition error at: '" = xyes; then\n    AC_DEFINE(ENABLE_S390_EXCESS_FLOAT_PRECISION, 1,\n[Define to enable evaluating float expressions with double precision in\nstandards-compatible mode on s390 targets.])\n  fi\n  ;;\nesac\n\n# Configure the subdirectories\n# AC_CONFIG_SUBDIRS($subdirs)\n\n# Create the Makefile\n# and configure language subdirectories\nAC_CONFIG_FILES($all_outputs)\n\nAC_CONFIG_COMMANDS([default],\n[\ncase ${CONFIG_HEADERS} in\n  *auto-host.h:config.in*)\n  echo > cstamp-h ;;\nesac\n# Make sure all the subdirs exist.\nfor d in $subdirs doc build common c-family\ndo\n    test -d $d || mkdir $d\ndone\n], \n[subdirs='$subdirs'])\nAC_OUTPUT\n\n'


In [6]:
Non_SATD_comments['codeSnippet'] = code_snippets

In [97]:
Non_SATD_comments.to_csv('../data/Non_SATD_clones_with_code.csv')

In [7]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import math
import time, pickle, math, warnings, os, operator
import string 
from sklearn.metrics.pairwise import cosine_similarity
from nltk.translate import bleu_score
import time

In [8]:
Non_SATD_comments_clone_same_system = Non_SATD_comments.loc[Non_SATD_comments['systemDiversity'] == 1].reset_index()
punctuations = string.punctuation.replace("\"","")
cleand_code_snippets = [code.replace('\n',' ').strip() for code in list(Non_SATD_comments_clone_same_system['codeSnippet'])]
cleand_code_snippets = [code.translate(str.maketrans({key: " {0} ".format(key) for key in punctuations})) for code in cleand_code_snippets]
Non_SATD_comments_clone_same_system['cleaned_code'] = cleand_code_snippets
data_count_vect = CountVectorizer(max_df=0.5)
data_vect = data_count_vect.fit_transform(cleand_code_snippets)

In [9]:
similarity = cosine_similarity(data_vect)

In [99]:
import itertools
cos_score_mins = []
cos_score_maxs = []
cos_score_means = []
cos_score_medians = []
systems = []
names = []
for name, group in Non_SATD_comments_clone_same_system.groupby('groupId'):
    group_index = list(group.index.values)
    cos_score = []
    names.append(name)
    for combination in itertools.combinations(group_index, 2):
        cos_score.append(similarity[combination[0]][combination[1]])
    systems.append(group.iloc[0]['buildSysteminFile'])
    stat = pd.DataFrame(cos_score)[0].describe()
    cos_score_mins.append(stat['min'])
    cos_score_maxs.append(stat['max'])
    cos_score_means.append(stat['mean'])
    cos_score_medians.append(stat['50%'])

In [100]:
stat_df = pd.DataFrame({'min': cos_score_mins,
             'max': cos_score_maxs,
             'mean': cos_score_means,
             'median': cos_score_medians,
            'system': systems,
            'groupId': names})

In [101]:
stat_df.to_csv("../data/RQ2_Non_SATD_stat.csv",index=None)